In [ ]:
import pypyjit
pypyjit.set_param('max_unroll_recursion=-1') # pypyの再帰高速化
import sys
sys.setrecursionlimit(10**9) # 再帰関数の呼び出し制限
from collections import defaultdict
from collections import deque
from heapq import *

class UnionFind:
    '''
    UnionFind
    n: 要素数 (0~n-1)
    parents: 各要素の親要素の番号を格納するリスト(要素が根の場合はそのグループの要素数を返す)
    find(x): 要素xが属するグループの根を返す
    union(x, y): 要素xが属するグループと要素yが属するグループとを併合する
    size(x): 要素xが属するグループのサイズ(要素数)を返す
    same(x, y): 要素x, yが同じグループに属するかどうかを返す
    members(x): 要素xが属するグループに属する要素をリストで返す
    roots(): 全ての根の要素をリストで返す
    group_count(): グループの数を返す
    all_group_members(): {根: そのグループに含まれる要素のリスト, ....}のdefaultdictを返す
    __str__(): print()での表示用
    '''
    
    def __init__(self, n):
        self.n = n
        self.parents = [-1] * n

    def find(self, x):
        if self.parents[x] < 0:
            return x
        else:
            self.parents[x] = self.find(self.parents[x])
            return self.parents[x]

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)

        if x == y:
            return
        
        if self.parents[x] > self.parents[y]:
            x, y = y, x

        self.parents[x] += self.parents[y]
        self.parents[y] = x

    def size(self, x):
        return -self.parents[self.find(x)]

    def same(self, x, y):
        return self.find(x) == self.find(y)

    def members(self, x):
        root = self.find(x)
        return [i for i in range(self.n) if self.find(i) == root]

    def roots(self):
        return [i for i, x in enumerate(self.parents) if x < 0]

    def group_count(self):
        return len(self.roots())

    def all_group_members(self):
        group_members = defaultdict(list)
        for member in range(self.n):
            group_members[self.find(member)].append(member)
        return group_members

    def __str__(self):
        return '\n'.join(f'{r}: {m}' for r, m in self.all_group_members().items())

def main():
    N,M = map(int,input().split())
    A = [0] + list(map(int,input().split()))
    uf = UnionFind(N+1)
    adj_dict = defaultdict(set)
    uv = []
    for _ in range(M):
        u,v = map(int,input().split())
        if A[u] == A[v]:
            uf.union(u,v)
        uv.append((u,v))
    # Aからの逆引き
    A_to_i = defaultdict(set)
    for root in uf.roots():
        A_to_i[A[root]].add(root)
    for u,v in uv:
        if A[u] < A[v]:
            adj_dict[uf.find(u)].add(uf.find(v))
        if A[u] > A[v]:
            adj_dict[uf.find(v)].add(uf.find(u))
    # A[i]が小さい順に距離を確定していく
    dist = defaultdict(int)
    dist[uf.find(1)] = 1
    for a in sorted(set(A)):
        if a < A[1]:
            continue
        for u in A_to_i[a]:
            if u not in adj_dict:
                continue
            if dist[u] == 0:
                continue
            for v in adj_dict[u]:
                if dist[v] < dist[u] + 1:
                    dist[v] = dist[u] + 1
    print(dist[uf.find(N)])

if __name__ == "__main__":
    main()